# Project Journal

This notebook serves to be a journal of the project. It shall present interesting and important results of the project.

### Dataset choice

The first task in this project is to choose the dataset and extract the appropriate data. There are several datasets that can be used for this project. The three main ones are presented below:

#### 1. Million Song Dataset
This dataset was presented by Thierry Bertin-Mahieux, Daniel P.W. Ellis, Brian Whitman, and Paul Lamere. *The Million Song Dataset. In Proceedings of the 12th International Society for Music Information Retrieval Conference (ISMIR 2011), 2011*.

This dataset has about 1,000,000 tracks from 44,745 different artists. The audio here is **not** directly accessibly. However, it can be obtained through 7digital API. Through contacting Lee et al. from *Sample-Level Deep Convolutional Neural Networks for Music Auto-Tagging using Raw Waveforms*, FTP access to all the files was achieved. However, this dataset ranges in the size of about **100TB** which is relatively unfeasible to download and use on a 3 month MSC project. This option is set aside to be used as backup if none of the other datasets are available to use.

#### 2. Magnatagatune Dataset
This dataset was presented by Edith Law, Kris West, Michael Mandel, Mert Bay and J. Stephen Downie. *Evaluation of algorithms using games: the case of music annotation. In  Proceedings of the 10th International Conference on Music Information Retrieval (ISMIR 2009), 2009*. 

This dataset has about 25,863 tracks from 230 different artists. In this dataset the audio is available. It can be seen that this dataset does not have a large variety of artists whilst being relatively small. However, this is the simplest to use as it has both the music and tags ready available. This is considered to be the primary option as a dataset.

#### 3. FMA dataset
This is a new dataset presented in April 2017 by Michaël Defferrard, Kirell Benzi, Pierre Vandergheynst and Xavier Bresson. *FMA: A Dataset for Music Analysis. In arXiv 2017.

This dataset has about 106,574 tracks from 16,341 different artists. It has audio available. However, in this case specific tags apart from the genre, era and artist name are not available. This are the tags that were actually needed for the task. This was seen as a better dataset from the Magnatagatune Dataset. Two methods were attempted to obtain the appropriate tags. 

##### 3.1 Method one Obtain tags through  Last.FM sqlite tag database
The tags from the Last.FM sqlite tag database were attempted to be extracted. These tags are partly used the MSD dataset. These were obtained both by hand and using the Last.FM API interface. However, in this database, there is no reference between the track id and the track name. This prove to be a problem since the only way of finding such a relationship was to sift through all the seperate JSON files and find the matching name to track id, and then extract the tags. This would be very computationally intensive since, there are ~100,000 songs in the FMA and the Last.FM database has ~500,000 tracks. Each specific track needs to be found from a folder with ~500,000 JSON files.

##### 3.2 Use Last.FM API
The second option was to use the Last.FM API. A function called lastfm_apireq.py was setup with a personal Last.FM account that can use API calls to ask for the tracks and extract the tags. However, from a sample of 100, more than 75% of the tracks had no tags. 
On thing that could be done instead is to check for album tags instead. However, these would not be track specific. 

Instead it was proposed that the FMA dataset is used prior to doing the tests using the MTAT dataset in order to find the difference in perfomance between the datasets. This could be done as shown by Lee et al. in *Sample-Level Deep Convolutional Neural Networks for Music Auto-Tagging using Raw Waveforms* when they compared the MTAT with MSD.

### Dataset Extraction

In order to extract the data from the magnatatatune dataset, a function extract_ds.py was created. This funciton does the following:

1. Extract metadata - Extract tags, mp3 filenames, track ids, and label map.
2. Shuffle data - Shuffle the data according to the seed provided in the package __init__.py file.
3. Reduction of samples - Reduce the number of files (if needed). If no reduction -1 is provided.
4. Sort tags - The targets and label map is sorted according to the frequency of tags.
5. Extract data - The data is extracted from the mp3 files and saved as numpy arrays in tracks folder.
6. Seperate and merge - Metadata is separated into 3 archives. Train, valid, test in fractions.
7. Save metadata archives - Metadata is saved as archives that can be loaded and tids used to load the numpy music arrays.

The extract_ds.py is provided in the pydst package. Logger logs info level information to ext_ds_XX_XX.log files in the log folder.

Total Runtime: **52:24**

Files that given error: **3** 

Reason: **The file is corrupted as it is 0 Bytes**

Total size of tracks folder: **~45GB**

### Data Provider

One of the problems of the data provider is that the dataset is considerably large. If the raw sample data is kept, this is about 45GB which is not possible to load in memory. If average pooling is performed over 10 samples, the dataset would reduce to 4.5GB which is still very large and not ideal to load in memory. 

One of the methods to solve this is to load batchwise. However, if this is done naively, there would be an additional computational cost which includes calling for the data before running the tensorflow session. Hence, the whole runs over all the epochs would require about (train_size/batch_size)*epochs number of disk calls. This is a considerably overhead.

The solution opted for this scenario is to use a FIFO and a dedicated thread to load the FIFO and the main thread to run the tensorflow graph.

In [ ]:
import tensorflow as tf
import numpy as np
import threading
import os

# Define constants (later to be converted to function params)
SAMPLE_SIZE = 1000
BATCH_SIZE = 20 
TARGET_SIZE = 189
LOAD_SIZE = 20
MAX_Q


trainq_din = tf.placeholder(tf.float32, shape=[LOAD_SIZE, SAMPLE_SIZE])
trainq_tin = tf.placeholder(tf.float32, shape=[LOAD_SIZE, TARGET_SIZE])
evalq_din = tf.placeholder(tf.float32, shape=[LOAD_SIZE, SAMPLE_SIZE])
evalq_tin = tf.placeholder(tf.float32, shape=[LOAD_SIZE, TARGET_SIZE])


train_q = tf.FIFOQueue(shapes=[[SAMPLE_SIZE],[TARGET_SIZE]], capacity=100, dtypes=[tf.float32, tf.float32])
eval_q = tf.FIFOQueue(shapes=[[SAMPLE_SIZE],[TARGET_SIZE]], capacity=100, dtypes=[tf.float32, tf.float32])

train_enqop = train_q. enqueue_many([trainq_din, trainq_tin])
train_deqop = train_q.dequeue()

eval_enqop = eval_q. enqueue_many([evalq_din, evalq_tin])
eval_deqop = eval_q.dequeue()

train_input_bx, train_target_bx = tf.train.batch(train_deqop, batch_size=BATCH_SIZE, capacity=100)
eval_input_bx, eval_target_bx = tf.train.batch(eval_deqop, batch_size=BATCH_SIZE, capacity=100)


def load_q(sess, q, MAX_Q, metad_path, data_dir, LOAD_SIZE):
    # Check number of files in train_dir for _folder_end
    metad = np.load(metad_path)
    maxp = metad.shape[0]
    lowp = 0
    
    while train_q.size < MAX_Q:
        topp = lowp + LOAD_SIZE
        
        if topp <= maxp:
            toload = metad['tids'][lowp:topp]
            ctargets = metad['targets'][lowp:topp, TARGET_SIZE]
            
            for idx, tid in enumerate(toload):
                loaded = np.load(data_dir + tid + '.npy')
                if idx = 1:
                    cdata = loaded
                else:
                    np.concatenate(cdata, loaded)
            lowp = topp
            
        else:
            ovrp = topp-maxp
            toload = np.concatenate(metad[lowp:maxp], metad[0:ovrp])
            ctargets = np.concatenate(metad['targets'][lowp:topp, TARGET_SIZE],
                                      metad['targets'][0:ovrp, TARGET_SIZE])
            
            for idx, tid in enumerate(toload):
                loaded = np.load(data_dir + tid + '.npy')
                if idx = 1:
                    cdata = loaded
                else:
                    np.concatenate(cdata, loaded)
            lowp = ovrp
        sess.run(q, feed_dict={din: cdata, tin:c target})

        
                
            
            
        
        
    
    
    



### Functions to be Designed

1. DenseLayer
2. conv1D
3. 
